In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../../../')

from notebooks.utils import load_node_features, get_referral_sites_edges, export_model_as_feature
from train import run_experiment

2022-02-27 00:22:23.518758: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load referral sites edges for level 1

In [2]:
level = 1
referral_sites_NODES = get_referral_sites_edges(data_year=2018, level=level)

print(referral_sites_NODES[:5])

02-27 00:22:23 notebooks.utils INFO     Processing level 0
02-27 00:22:23 notebooks.utils INFO     Node length: 3922
02-27 00:22:23 notebooks.utils INFO     Distinct node length: 3922
02-27 00:22:23 notebooks.utils INFO     Processing level 1
02-27 00:22:23 notebooks.utils INFO     Node length: 11144
02-27 00:22:23 notebooks.utils INFO     Distinct node length: 11144


[('rantt.com', 'anxiousminds.org'), ('plymouthpatriotpost.com', 'plymouthpatriotpost.com'), ('antifascistnews.net', 'angrywhitemen.org'), ('oneindia.com', 'raftaar.in'), ('columbiadailyherald.com', 'springhilltn.org')]


In [3]:
edge_df = pd.DataFrame(referral_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,rantt.com,anxiousminds.org
1,plymouthpatriotpost.com,plymouthpatriotpost.com
2,antifascistnews.net,angrywhitemen.org
3,oneindia.com,raftaar.in
4,columbiadailyherald.com,springhilltn.org


### Find all unique nodes in edges

In [4]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 8250 Sample: ['anh.com', 'nrawomen.tv', 'evaluationtoolbox.net.au', 'guyhut.com', 'holodomorinfo.com']


### 1. Load all node features

In [5]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [6]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [7]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8250 entries, anh.com to animalmatters.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   6466 non-null   float64
 1   daily_pageviews_per_visitor  6467 non-null   float64
 2   daily_time_on_site           4976 non-null   float64
 3   total_sites_linking_in       8110 non-null   float64
 4   bounce_rate                  4667 non-null   float64
dtypes: float64(5)
memory usage: 386.7+ KB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [8]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8250 entries, anh.com to animalmatters.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   8250 non-null   float64
 1   daily_pageviews_per_visitor  6467 non-null   float64
 2   daily_time_on_site           4976 non-null   float64
 3   total_sites_linking_in       8250 non-null   float64
 4   bounce_rate                  4667 non-null   float64
dtypes: float64(5)
memory usage: 386.7+ KB


### 3. Normalizing features

In [9]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [10]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 8250, Edges: 15066

 Node types:
  default: [8250]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [15066]
        Weights: all 1 (default)
        Features: none


# Unsupervised GraphSAGE
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

In [11]:
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 128
epochs = 4
num_samples = [10, 5]

graphsage_link_generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_graphsage_link_gen = graphsage_link_generator.flow(unsupervised_samples)

layer_sizes = [128, 512]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=graphsage_link_generator, bias=True, dropout=0.0, normalize="l2"
)

# Build the model and expose input and output sockets of graphsage, for node pair inputs:
x_inp, x_out = graphsage.in_out_tensors()

prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

graphsage_model = keras.Model(inputs=x_inp, outputs=prediction)

graphsage_model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.categorical_crossentropy,
    metrics=['acc'],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [12]:
history = graphsage_model.fit(
    train_graphsage_link_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

Epoch 1/4
516/516 [==============================] - 64s 121ms/step - loss: 5.9605e-08 - acc: 0.5002
Epoch 2/4
516/516 [==============================] - 61s 118ms/step - loss: 5.9605e-08 - acc: 0.5002
Epoch 3/4
516/516 [==============================] - 67s 128ms/step - loss: 5.9605e-08 - acc: 0.5003
Epoch 4/4
516/516 [==============================] - 64s 123ms/step - loss: 5.9605e-08 - acc: 0.5002


In [13]:
embedding_model = keras.Model(inputs=x_inp[0::2], outputs=x_out[0])

from stellargraph.mapper import GraphSAGENodeGenerator

node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_features_df.index)
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

# print('Sample:', embeddings_wv['crooked.com'][:10])

65/65 [==============================] - 4s 58ms/step


# Export embeddings as feature

In [14]:
export_model_as_feature(embeddings_wv, f'graph_sage_referral_sites_level_{level}_epochs_{epochs}', data_year='2018')

'/Users/panayot/Documents/News-Media-Peers/data/emnlp2018/features/graph_sage_referral_sites_level_1_epochs_4.json'

In [15]:
run_experiment(features=f'graph_sage_referral_sites_level_{level}_epochs_{epochs}', dataset='emnlp2018')

+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| fact | emnlp2018 |  single classifier  |    combine    |       False        | graph_sage_referral_sites_level_1_epochs_4 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+


02-27 00:26:57 train        INFO     Start training...
02-27 00:26:57 train        INFO     Fold: 0
02-27 00:27:13 train        INFO     Fold: 1
02-27 00:27:26 train        INFO     Fold: 2
02-27 00:27:39 train        INFO     Fold: 3
02-27 00:27:52 train        INFO     Fold: 4


+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+--------------------+-------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |      Macro-F1      |      Accuracy     |  Flip error-rate   |        MAE        |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+--------------------+-------------------+
| fact | emnlp2018 |  single classifier  |    combine    |       False        | graph_sage_referral_sites_level_1_epochs_4 | 28.271276285077107 | 51.74363807728558 | 22.431668237511783 | 0.706880301602262 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+-------

In [16]:
run_experiment(features=f'graph_sage_referral_sites_level_{level}_epochs_{epochs}', dataset='emnlp2018', task='bias')

+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| bias | emnlp2018 |  single classifier  |    combine    |       False        | graph_sage_referral_sites_level_1_epochs_4 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+


02-27 00:28:06 train        INFO     Start training...
02-27 00:28:06 train        INFO     Fold: 0
02-27 00:28:21 train        INFO     Fold: 1
02-27 00:28:33 train        INFO     Fold: 2
02-27 00:28:47 train        INFO     Fold: 3
02-27 00:28:57 train        INFO     Fold: 4


+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |      Macro-F1     |      Accuracy     |  Flip error-rate   |        MAE         |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| bias | emnlp2018 |  single classifier  |    combine    |       False        | graph_sage_referral_sites_level_1_epochs_4 | 26.58105927842221 | 52.30914231856739 | 1.0367577756833177 | 0.4872761545711593 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+-------------------+--------